In [ ]:
!pip3 install pandas
!pip3 install numpy
!pip3 install scikit-learn

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression

In [ ]:
wind_ava = pd.read_csv('wind_ava.csv.gz', compression="gzip")

# Elimina todas las variables meteorológicas que no correspondan a la localización de Sotavento (la localización 13)
wind_ava = wind_ava.filter(regex='^(datetime|energy|.*\.13)$')

# Elimina las variables 'lai_hv.13' y 'v10n.13'
if 'lai_hv.13' in wind_ava.columns and 'v10n.13' in wind_ava.columns:
    wind_ava = wind_ava.drop(['lai_hv.13', 'v10n.13'], axis=1)

# Vuelve a definir las variables numéricas
numerical_vars = wind_ava.select_dtypes(include=['int64', 'float64']).columns

# Quitamos "energy" de las variables numéricas, ya que querremos usarla como variable objetivo
numerical_vars = numerical_vars.drop('energy')

# Dividir los datos en conjuntos de entrenamiento y prueba (evaluación externa)
X_train, X_test, y_train, y_test = train_test_split(wind_ava[numerical_vars], wind_ava['energy'], test_size=1/3, random_state=100472166, shuffle=False)

# Transforma los datos de entrenamiento y de competición con el método de escalado
X_train_scaled = MinMaxScaler.transform(X_train[numerical_vars])

# Apartado 5

En resumen, el método elegido es el de ``regresión lineal`` y la técnica de escalado es ``MinMaxScaler``. Estos serán por lo tanto los métodos que utilicemos para realizar la evalución del rendimiento futuro, como se muestra en el siguiente código:

In [ ]:
import pickle

# Carga los datos de competición en un DataFrame
competition_data = pd.read_csv('wind_comp.csv.gz', compression="gzip")

# Ajusta el escalador a tus datos de entrenamiento
scaler = MinMaxScaler()
scaler.fit(X_train[numerical_vars])

# Transforma los datos de entrenamiento y de competición con el método de escalado
X_train_scaled = scaler.transform(X_train[numerical_vars])
X_competition_scaled = scaler.transform(competition_data[numerical_vars])

# Selecciona el mejor modelo que fue la Regresión Lineal
best_model = LinearRegression()

# Entrena el modelo final con todos los datos
best_model.fit(X_train_scaled, y_train)

# Guarda el modelo final
with open('modelo_final.pkl', 'wb') as file:
    pickle.dump(best_model, file)

# Carga el modelo final
with open('modelo_final.pkl', 'rb') as file:
    loaded_model = pickle.load(file)

# Utiliza el modelo final para hacer predicciones en el conjunto de datos de la competición y las guarda en un CSV
competition_predictions = loaded_model.predict(X_competition_scaled)
pd.DataFrame(competition_predictions).to_csv('predicciones.csv', index=False)